In [2]:
import csv
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib
import re
import matplotlib.gridspec as gridspec
import scipy.signal as signal

import seaborn as sns
sns.set(color_codes=True)

#matplotlib.style.use('ggplot')
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

height has been deprecated.



# Event files pre-processing

In [2]:
def modifierfun (row):
    if (pd.notnull(row['Modifier_1'])):
        return row['Modifier_1'];
    if (pd.notnull(row['Modifier_2'])):
        return row['Modifier_2'];
    if (pd.notnull(row['Modifier_3'])):
        return row['Modifier_3'];

def get_sec(row):
    s=row['Time_Absolute_hms']
    l = s.split(':')
    return int(l[0]) * 3600 + int(l[1]) * 60 + int(l[2])

## Generate event related dataframe from ObserverXT data

In [3]:
dfAll=pd.DataFrame()
for fn in os.listdir('./../observerXT/'):
    l=fn.split(' - ')
    t=l[1].split('_')
    sujet=int(t[0][1:])
    matnum=t[1][-1]
    matname=t[2]    
    niveau={
        'debutant':1,
        'amateur':2,
        'semipro':3,
        'pro':4,
        'champion':5,
        'legend':6
    }[matname]
    eventfile='./../observerXT/'+fn
    #print eventfile
    df=pd.read_csv(eventfile,sep=';', quoting=csv.QUOTE_ALL,encoding='utf-16')
    df = df[df.Event_Type == "State start"]
    df['Absolute_sec']=df.apply (lambda row: get_sec(row),axis=1)
    df['annovalue']=df.apply (lambda row: modifierfun(row),axis=1)
    df=df[['Time_Absolute_hms','Time_Relative_hmsf','Absolute_sec','Behavior','annovalue']]
    
    ndf=df.pivot(index='Absolute_sec', columns='Behavior', values='annovalue')
    ndf=ndf[['event','emotions','arousal','valence']]
    ndf=ndf.reset_index()
    ndf.loc[:,'sujet']=sujet
    ndf.loc[:,'match']=matnum
    ndf.loc[:,'niveau']=niveau
    dfAll=dfAll.append(ndf,ignore_index=True)
dfAll=dfAll.sort(['sujet', 'match','Absolute_sec'], ascending=[1,1,1])
dfAll['sujet']=dfAll['sujet'].astype(int)
#dfAll.info() #datatype of each column
dfAll=dfAll.sort(['sujet','match','Absolute_sec']).reset_index(drop=True)
dfAll.to_csv('./../out/Allevent.txt',columns=['sujet','match','Absolute_sec','event','emotions','arousal','valence'],encoding='utf-16')

In [46]:
dfAll.ix[dfAll['sujet']==34,:]

,sujet,match,Absolute_sec,event,emotions,arousal,valence
1077,34,1,38010,arret du gardien,frustration,2,1
1078,34,1,38095,arret du gardien,joie,2,2
1079,34,1,38206,tir,frustration,1,-2
1080,34,1,38262,tir,joie,2,1
1081,34,1,38485,rate,frustration,0,-2
1082,34,1,38532,tir,peur,3,0
1083,34,1,38560,arret du gardien,frustration,3,1
1084,34,1,38598,tir,frustration,3,0
1085,34,1,38643,rate,frustration,3,0
1086,34,1,38666,arret du gardien,frustration,3,1


In [125]:
dfAll.head()

Behavior,Absolute_sec,event,emotions,arousal,valence,sujet,match,niveau
0,47637,rate,frustration,2,-1,1,1,2
1,47813,arret du gardien,colere,2,-1,1,1,2
2,47852,but,colere,2,-3,1,1,2
3,48157,tir,peur,1,-1,1,1,2
4,49632,arret du gardien,frustration,2,-2,1,2,1


## Analyse event emotion dataframe

#### Number of event ####

In [87]:
dfAll.pivot_table(values='event',index='sujet',columns='match',aggfunc=len).plot(stacked=True,kind='barh',fontsize=8)
plt.ylabel('subject')
plt.xlabel('Number of events')
#plt.title('Number of event annoted by each subject')
plt.savefig('numevent.png', bbox_inches='tight')

In [91]:
dfAll[['match','event']].groupby(['match']).count().plot(kind='bar',rot=30,legend=False)
plt.ylabel('Number')
plt.xlabel('Match')
#plt.title('Number of emotion annoted by each subject')
#plt.show()
plt.savefig('Anomatchnum.png', bbox_inches='tight')

#### Emotion / participant

In [94]:
dfAll.pivot_table(values='event',index='sujet',columns='emotions',aggfunc=len).plot(stacked=True,kind='barh',fontsize=8)
plt.ylabel('subject')
plt.xlabel('Number of emotions')
#plt.title('Number of emotions annoted by each subject')
plt.show()

In [95]:
dfAll.pivot_table(values='event',index='sujet',columns='arousal',aggfunc=len).plot(stacked=True,kind='barh',fontsize=8)
plt.ylabel('subject')
plt.xlabel('Number of emotion - arousal')
#plt.title('Number of emotion - arousal annoted by each subject')
plt.show()

In [96]:
dfAll.pivot_table(values='event',index='sujet',columns='valence',aggfunc=len).plot(stacked=True,kind='barh',fontsize=8)
plt.ylabel('subject')
plt.xlabel('Number of emotion -valence')
#plt.title('Number of emotion - valence annoted by each subject')
plt.show()

#### Emotion / all

In [32]:
dfAll[['emotions','event']].groupby(['emotions']).count().sort('event',ascending=False).plot(kind='bar',rot=30,legend=False)
plt.ylabel('Number')
plt.xlabel('Emotion')
#plt.title('Number of emotion annoted by each subject')
#plt.show()
plt.savefig('Numemo.png', bbox_inches='tight')

In [88]:
dfAll[['arousal','event']].groupby(['arousal']).count().plot(kind='bar',legend=False)
plt.ylabel('Number')
plt.xlabel('Arousal')
#plt.title('Number of emotion (arousal) annoted by each subject')
plt.show()

In [89]:
dfAll[['valence','event']].groupby(['valence']).count().plot(kind='bar',legend=False)
plt.ylabel('Number')
plt.xlabel('emotion')
#plt.title('Number of emotion (valence) annoted by each subject')
plt.show()

In [10]:
mean, cov = [0, 1], [(1, .5), (.5, 1)]
data = np.random.multivariate_normal(mean, cov, 200)
df = pd.DataFrame(data, columns=["x", "y"])

In [26]:
fig=sns.jointplot(x="arousal", y="valence", data=dfAll, kind="hex", stat_func=None,xlim=(-3.5,3.5),ylim=(-3.5,3.5))
#plt.show()
fig.savefig('AVDis.png', bbox_inches='tight')

In [91]:
tmp=dfAll.as_matrix(columns=['arousal','valence'])
dfAll[dfAll.isnull().any(axis=1)]

Behavior,Absolute_sec,event,emotions,arousal,valence,sujet,match,niveau


In [33]:
import numpy as np
import numpy.random
import matplotlib.pyplot as plt
tmp=dfAll.as_matrix(columns=['arousal','valence'])
x = np.int_(tmp[:,0])
y = np.int_(tmp[:,1])
heatmap, xedges, yedges = np.histogram2d(x, y,bins=[10,10])
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.imshow(heatmap, extent=extent, clim=(0.0, 170))
plt.show()

In [34]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter

# the random data
tmp=dfAll.as_matrix(columns=['arousal','valence'])
x = np.int_(tmp[:,0])
y = np.int_(tmp[:,1])
heatmap, xedges, yedges = np.histogram2d(x, y,bins=[7,7])
extent = [-3.5, 3.5, -3.5, 3.5]

nullfmt = NullFormatter()         # no labels

# definitions for the axes
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
bottom_h = left_h = left + width + 0.02

rect_scatter = [left, bottom, width, height]

rect_histx = [left, bottom_h, width, 0.2]
rect_histy = [left_h, bottom, 0.2, height]

# start with a rectangular Figure
plt.figure(1, figsize=(8, 8))


axScatter = plt.axes(rect_scatter)
axHistx = plt.axes(rect_histx)
axHisty = plt.axes(rect_histy)

# no labels
axHistx.xaxis.set_major_formatter(nullfmt)
axHisty.yaxis.set_major_formatter(nullfmt)

# the scatter plot:
axScatter.imshow(heatmap, extent=extent, cmap="hot")


# now determine nice limits by hand:
binwidth = 1
xymax = 3.5
lim = 3.5

axScatter.set_xlim((-lim, lim))
axScatter.set_ylim((-lim, lim))

bins = np.arange(-lim, lim + binwidth, binwidth)
axHistx.hist(x, bins=bins)
axHisty.hist(y, bins=bins, orientation='horizontal')

axHistx.set_xlim(axScatter.get_xlim())
axHisty.set_ylim(axScatter.get_ylim())

plt.show()

## Select event files for analysis

In [94]:
selectdf=dfAll[dfAll.valence>0]
for i in range(len(selectdf)):
    chunk=selectdf.iloc[i,5]+'_'+str(selectdf.index[i])+'.txt'
    print chunk


1_14.txt
2_20.txt
2_26.txt
2_36.txt
2_37.txt
2_44.txt
2_50.txt
2_62.txt
3_71.txt
3_76.txt
3_78.txt
3_81.txt
3_85.txt
3_93.txt
3_94.txt
3_100.txt
3_102.txt
3_105.txt
3_107.txt
4_112.txt
4_114.txt
4_117.txt
4_118.txt
4_120.txt
4_121.txt
4_122.txt
4_125.txt
4_128.txt
4_130.txt
4_131.txt
4_133.txt
4_140.txt
4_141.txt
4_142.txt
5_147.txt
5_148.txt
5_149.txt
5_155.txt
5_156.txt
5_157.txt
5_162.txt
5_165.txt
5_168.txt
5_172.txt
5_174.txt
5_175.txt
5_179.txt
6_190.txt
6_192.txt
6_193.txt
6_202.txt
6_204.txt
7_207.txt
7_208.txt
7_212.txt
7_219.txt
7_220.txt
8_225.txt
8_229.txt
8_231.txt
8_232.txt
8_234.txt
8_238.txt
9_247.txt
9_255.txt
10_266.txt
10_267.txt
10_269.txt
10_272.txt
10_273.txt
10_275.txt
10_280.txt
10_282.txt
10_283.txt
10_284.txt
10_286.txt
10_290.txt
10_303.txt
11_306.txt
11_307.txt
11_309.txt
11_312.txt
11_316.txt
11_318.txt
11_321.txt
11_323.txt
11_325.txt
11_326.txt
11_327.txt
12_330.txt
12_335.txt
12_338.txt
12_341.txt
12_342.txt
12_347.txt
12_349.txt
12_352.txt
12_355.txt
12

In [14]:

#chunk='Data/match1.txt'
chunk='S1_40.txt'
chunkdf=pd.read_csv(chunk,sep='\t',nrows=30000,header=None,usecols=range(1,12),names=['rsp', 'ppg', 'x', 'emg_s','skt','y','eda','ecg','z','emg_f','temp'])
#chunkdf=pd.read_csv(chunk,sep='\t',header=None,nrows=60000*13.8,usecols=range(1,14),names=['rsp', 'ppg', 'HR', 'x', 'emg_s','skt','RR','y','eda','ecg','z','emg_f','temp'])



## Generate event related signal sequence from Biopac data

#### Read event file

In [5]:
dfAll = pd.read_csv('./../out/Allevent.txt',encoding='utf-16',index_col=0)

In [6]:
dfAll.head()

,sujet,match,Absolute_sec,event,emotions,arousal,valence
0,1,1,47637,rate,frustration,2,-1
1,1,1,47813,arret du gardien,colere,2,-1
2,1,1,47852,but,colere,2,-3
3,1,1,48157,tir,peur,1,-1
4,1,2,49632,arret du gardien,frustration,2,-2


In [8]:
winsize=5
biopacfilels=[]
for fn in os.listdir('./../biopac/'):
    biopacfilels.append(fn)

for i in range(len(dfAll)):
    #print i
    eventpoint=dfAll.iloc[i,2]
    ebegin=eventpoint-winsize
    eend=eventpoint+winsize
    sujet=dfAll.iloc[i,0]
    eidx = dfAll.index[i]
    #Biopacfile
    patternbiopacfile="^S"+str(sujet)+"_.*"
    regexb=re.compile(patternbiopacfile)
    bfilel=[m.group(0) for l in biopacfilels for m in [regexb.search(l)] if m]
    if len(bfilel)==1:
        bfile = './../biopac/'+bfilel[0]
        efile = './../biopacEvent/S_'+str(sujet)+'_'+str(eidx)+"_.txt"
        cmd_biopac = "sed -n '/^" + str(ebegin) + "/,/^" + str(eend) + "/p' "+bfile+">"+efile
        print cmd_biopac
        #os.system(cmd_biopac)

    
       

sed -n '/^38399/,/^38409/p' ./../biopac/S2_2016-05-11T10_16_44.txt>./../biopacEvent/S_2_18_.txt
sed -n '/^38490/,/^38500/p' ./../biopac/S2_2016-05-11T10_16_44.txt>./../biopacEvent/S_2_19_.txt
sed -n '/^38608/,/^38618/p' ./../biopac/S2_2016-05-11T10_16_44.txt>./../biopacEvent/S_2_20_.txt
sed -n '/^38846/,/^38856/p' ./../biopac/S2_2016-05-11T10_16_44.txt>./../biopacEvent/S_2_21_.txt
sed -n '/^38875/,/^38885/p' ./../biopac/S2_2016-05-11T10_16_44.txt>./../biopacEvent/S_2_22_.txt
sed -n '/^38903/,/^38913/p' ./../biopac/S2_2016-05-11T10_16_44.txt>./../biopacEvent/S_2_23_.txt
sed -n '/^38936/,/^38946/p' ./../biopac/S2_2016-05-11T10_16_44.txt>./../biopacEvent/S_2_24_.txt
sed -n '/^38984/,/^38994/p' ./../biopac/S2_2016-05-11T10_16_44.txt>./../biopacEvent/S_2_25_.txt
sed -n '/^39045/,/^39055/p' ./../biopac/S2_2016-05-11T10_16_44.txt>./../biopacEvent/S_2_26_.txt
sed -n '/^40465/,/^40475/p' ./../biopac/S2_2016-05-11T10_16_44.txt>./../biopacEvent/S_2_27_.txt
sed -n '/^40492/,/^40502/p' ./../biopac/

## Tobbi data

In [340]:
tobiidf=pd.read_csv('/home/wenlu/Documents/XPFIFA/tobii/newTobiiFile2/S33.tsv',sep='\t',header=None,names=['ts','LP', 'RP'])
#tobiidf.head()
#tobiidf[tobiidf.isnull().any(axis=1)]
dftmp=tobiidf.ix[(tobiidf.ts >38822)&(tobiidf.ts <39062),1:3]

plt.figure(); dftmp.plot()
plt.show()

In [2]:
tobiifilels=[]
for fn in os.listdir('./../tobii/newTobiiFile2/'):
    tobiifilels.append(fn)
tobiifilels

#for fn in os.listdir('./../tobii/files'):
    #tobiifilels.append(fn)
for i in range(len(dfAll)):
    #print i
    eventpoint=dfAll.iloc[i,0]
    ebegin=eventpoint-15
    eend=eventpoint+15
    sujet=dfAll.iloc[i,5]
    eidx = dfAll.index[i]
    #print eend
    patterntobiifile="^S"+str(sujet)+".*"
    regexb=re.compile(patterntobiifile)
    tfilel=[m.group(0) for l in tobiifilels for m in [regexb.search(l)] if m]
    if len(tfilel)==1:
        tfile = './../tobii/newTobiiFile2/'+tfilel[0]
#         cmd = "awk -F '\t' 'if(NR==1) print $1' "+tfile
#         headts = os.system(cmd)
#         print headts
#         cmd = "awk -F '\t' 'END{print $1}' "+tfile
#         tailts = os.system(cmd)
#         print tailts
#         if headts>ebegin:
#             ebegin=headts
#             print 'Begin changed'
#         if tailts<eend:
#             eend =tailts
#             print 'End changed'
        efile = './../tobiiEvent/S_'+str(sujet)+'_'+str(eidx)+"_.txt"
        cmd = "sed -n '/^" + str(ebegin) + "/,/^" + str(eend) + "/p' "+tfile+">"+efile
        os.system(cmd)
        print cmd
        
    #Tobiifile
    
    #Facereaderfile


NameError: name 'os' is not defined

# Match Journal processing

In [5]:
def get_sec(row,col):
    s=row[col]
    l = s.split(':')
    return int(l[0]) * 3600 + int(l[1]) * 60 + int(l[2])

Mdf={}
Datapath='./../Matchjrn/'
Matchfilepath='./../biopacMatch/'
Biofilepath="./../biopac/"
biofilelist=[]
for fn in os.listdir(Biofilepath):
    biofilelist.append(fn)

num=0
for fn in os.listdir(Datapath):
    num=num+1
    sujet = re.split(r'\s*[.\-\s]\s*', fn)[1]  # M - S2.csv
    print num, sujet
    matchfile = Datapath + fn
    #print matchfile
    Mdfname=sujet
    df=pd.read_csv(matchfile,sep=',',header=None,index_col=0)
    df[3]=df.apply (lambda row: get_sec(row,1),axis=1)
    df[4]=df.apply (lambda row: get_sec(row,2),axis=1)
    df[5]=df[4]-df[3]
    df=df.loc[['base0','en1','en2','base1','m11','m12','base2','m21','m22','base3','m31','m32'],:]
    df.loc[['base0','base1','base2','base3'],6]=df.loc[:,4]-180
    df.loc[['en1','en2','m11','m12','m21','m22','m31','m32'],6]=df.loc[:,4]-240
    df[6]=df[6].astype(int)
    #df_o=df[[6,4]]
    Mdf[Mdfname]=df
    #df=pd.read_csv(matchfile,sep=',',header=None,index_col=0)
    for i in range(len(df)):
        mfile=Matchfilepath+sujet+'_'+str(df.index[i])+'.txt'
        #print mfile
        mbegin=df.ix[i,6]
        mend=df.ix[i,4]
        for j in biofilelist:
            if j.split('_')[0]==sujet:
                biopacfile=j
        cmd = "sed -n '/^" + str(mbegin) + "/,/^" + str(mend) + "/p' "+"./../biopac/"+ biopacfile + ">"+mfile
        print cmd
            #os.system(cmd)

1 S18
sed -n '/^48947/,/^49127/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_base0.txt
sed -n '/^49198/,/^49438/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_en1.txt
sed -n '/^49573/,/^49813/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_en2.txt
sed -n '/^49970/,/^50150/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_base1.txt
sed -n '/^50250/,/^50490/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_m11.txt
sed -n '/^50645/,/^50885/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_m12.txt
sed -n '/^51947/,/^52127/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_base2.txt
sed -n '/^52203/,/^52443/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_m21.txt
sed -n '/^52593/,/^52833/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_m22.txt
sed -n '/^53889/,/^54069/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_base3.txt
sed -n '/^54177/

In [45]:
Mdf['S2'] #Mdf['S11'].ix['base0',6]

,1,2,3,4,5,6
base0,10:19:50,10:23:10,37190,37390,200,37210
en1,10:23:18,10:28:46,37398,37726,328,37486
en2,10:28:46,10:33:21,37726,38001,275,37761
base1,10:35:30,10:38:50,38130,38330,200,38150
m11,10:38:55,10:43:59,38335,38639,304,38399
m12,10:46:11,10:51:09,38771,39069,298,38829
base2,11:10:00,11:13:10,40200,40390,190,40210
m21,11:13:17,11:18:49,40397,40729,332,40489
m22,11:20:19,11:25:24,40819,41124,305,40884
base3,11:42:30,11:45:50,42150,42350,200,42170


## Extract baseline sequence 

In [55]:
Matchfilepath='./../biopacMatch/'
basefilepath='./../biopacBase20/'
winsize = 20
f=1000
for sujet in range(1,59):
    #select last 'winsize' sec: [120, (120+winsize)], in the base0 file
    basefile = Matchfilepath+'S'+str(sujet)+'_base1.txt'
    ebegin=120*f
    eend=(120+winsize)*f
    ofile= basefilepath+'S_'+str(sujet)+'_base_'+str(winsize)+'_.txt'
    cmd_base = "awk 'NR >= "+str(ebegin)+" && NR <= "+str(eend)+"' "+ basefile + '>' +ofile
    print cmd_base
    os.system(cmd_base)

awk 'NR >= 120000 && NR <= 140000' ./../biopacMatch/S1_base1.txt>./../biopacBase20/S_1_base_20_.txt
awk 'NR >= 120000 && NR <= 140000' ./../biopacMatch/S2_base1.txt>./../biopacBase20/S_2_base_20_.txt
awk 'NR >= 120000 && NR <= 140000' ./../biopacMatch/S3_base1.txt>./../biopacBase20/S_3_base_20_.txt
awk 'NR >= 120000 && NR <= 140000' ./../biopacMatch/S4_base1.txt>./../biopacBase20/S_4_base_20_.txt
awk 'NR >= 120000 && NR <= 140000' ./../biopacMatch/S5_base1.txt>./../biopacBase20/S_5_base_20_.txt
awk 'NR >= 120000 && NR <= 140000' ./../biopacMatch/S6_base1.txt>./../biopacBase20/S_6_base_20_.txt
awk 'NR >= 120000 && NR <= 140000' ./../biopacMatch/S7_base1.txt>./../biopacBase20/S_7_base_20_.txt
awk 'NR >= 120000 && NR <= 140000' ./../biopacMatch/S8_base1.txt>./../biopacBase20/S_8_base_20_.txt
awk 'NR >= 120000 && NR <= 140000' ./../biopacMatch/S9_base1.txt>./../biopacBase20/S_9_base_20_.txt
awk 'NR >= 120000 && NR <= 140000' ./../biopacMatch/S10_base1.txt>./../biopacBase20/S_10_base_20_.tx

## Extract random baseline sequence

In [4]:
dfAll = pd.read_csv('./../out/Allevent.txt',encoding='utf-16',index_col=0)
nbevent= dfAll.groupby('sujet')['arousal'].count()

In [9]:
from random import randint
winsize = 20
f=1000
Matchfilepath='./../biopacMatch/'
randombasefilepath='./../biopacBase20_random/'
matchfilels=[]
for fn in os.listdir(Matchfilepath):
    matchfilels.append(fn)
    #Basefile
for sujet in np.arange(1,59):
    countbase=0
    patternbasefile="^S"+str(sujet)+"_base.*"
    regexb=re.compile(patternbasefile)
    basefilel=[m.group(0) for l in matchfilels for m in [regexb.search(l)] if m]
    if len(basefilel)==0:
        continue
    else:
        for baseseqn in np.arange(nbevent[sujet]):
            countbase=countbase+1
            basenum=randint(0,len(basefilel)-1)
            basefile = Matchfilepath+'S'+str(sujet)+'_base'+str(basenum)+'.txt'
            randombegin=randint(0,150)
            ebegin=randombegin*f
            eend=(randombegin+winsize)*f
            ofile= randombasefilepath+'S_'+str(sujet)+'_base_'+str(winsize)+'_'+str(countbase)+'.txt'
            cmd_base = "awk 'NR >= "+str(ebegin)+" && NR <= "+str(eend)+"' "+ basefile + '>' +ofile
            print cmd_base
            os.system(cmd_base)


awk 'NR >= 30000 && NR <= 50000' ./../biopacMatch/S2_base2.txt>./../biopacBase20_random/S_2_base_20_1.txt
awk 'NR >= 44000 && NR <= 64000' ./../biopacMatch/S2_base2.txt>./../biopacBase20_random/S_2_base_20_2.txt
awk 'NR >= 24000 && NR <= 44000' ./../biopacMatch/S2_base1.txt>./../biopacBase20_random/S_2_base_20_3.txt
awk 'NR >= 94000 && NR <= 114000' ./../biopacMatch/S2_base2.txt>./../biopacBase20_random/S_2_base_20_4.txt
awk 'NR >= 94000 && NR <= 114000' ./../biopacMatch/S2_base0.txt>./../biopacBase20_random/S_2_base_20_5.txt
awk 'NR >= 116000 && NR <= 136000' ./../biopacMatch/S2_base2.txt>./../biopacBase20_random/S_2_base_20_6.txt
awk 'NR >= 137000 && NR <= 157000' ./../biopacMatch/S2_base0.txt>./../biopacBase20_random/S_2_base_20_7.txt
awk 'NR >= 50000 && NR <= 70000' ./../biopacMatch/S2_base0.txt>./../biopacBase20_random/S_2_base_20_8.txt
awk 'NR >= 25000 && NR <= 45000' ./../biopacMatch/S2_base3.txt>./../biopacBase20_random/S_2_base_20_9.txt
awk 'NR >= 13000 && NR <= 33000' ./../bi

## Extract before event sequence

In [48]:
dfAll.head(20)

,sujet,match,Absolute_sec,event,emotions,arousal,valence
0,1,1,47637,rate,frustration,2,-1
1,1,1,47813,arret du gardien,colere,2,-1
2,1,1,47852,but,colere,2,-3
3,1,1,48157,tir,peur,1,-1
4,1,2,49632,arret du gardien,frustration,2,-2
5,1,2,49661,tir,peur,1,-1
6,1,2,49706,rate,frustration,1,-1
7,1,2,50038,faute,neutre,0,0
8,1,2,50049,but,colere,2,-2
9,1,2,50134,rate,frustration,2,-1


In [56]:
dfAll.shape

(1748, 7)

In [91]:
dfAll = pd.read_csv('./../out/Allevent.txt',encoding='utf-16',index_col=0)
winsize=14
biopacfilels=[]
for fn in os.listdir('./../biopac/'):
    biopacfilels.append(fn)

for i in range(len(dfAll)):
    #print i
    eventpoint=dfAll.iloc[i,2]
    #ebegin=eventpoint-winsize/2
    #eend=eventpoint+winsize/2
    beforeeventpoint=eventpoint-winsize/2
    ebegin=beforeeventpoint-winsize/2
    eend=beforeeventpoint+winsize/2
    sujet=dfAll.iloc[i,0]
    eidx = dfAll.index[i]
    #Biopacfile
    patternbiopacfile="^S"+str(sujet)+"_.*"
    regexb=re.compile(patternbiopacfile)
    bfilel=[m.group(0) for l in biopacfilels for m in [regexb.search(l)] if m]
    if len(bfilel)==1:
        bfile = './../biopac/'+bfilel[0]
        efile = './../biopacBeforeEvent14s/S_'+str(sujet)+'_'+str(eidx)+"_.txt"
        cmd_biopac = "sed -n '/^" + str(ebegin) + "/,/^" + str(eend) + "/p' "+bfile+">"+efile
        print cmd_biopac
        os.system(cmd_biopac)

sed -n '/^38390/,/^38404/p' ./../biopac/S2_2016-05-11T10_16_44.txt>./../biopacBeforeEvent14s/S_2_18_.txt
sed -n '/^38481/,/^38495/p' ./../biopac/S2_2016-05-11T10_16_44.txt>./../biopacBeforeEvent14s/S_2_19_.txt
sed -n '/^38599/,/^38613/p' ./../biopac/S2_2016-05-11T10_16_44.txt>./../biopacBeforeEvent14s/S_2_20_.txt
sed -n '/^38837/,/^38851/p' ./../biopac/S2_2016-05-11T10_16_44.txt>./../biopacBeforeEvent14s/S_2_21_.txt
sed -n '/^38866/,/^38880/p' ./../biopac/S2_2016-05-11T10_16_44.txt>./../biopacBeforeEvent14s/S_2_22_.txt
sed -n '/^38894/,/^38908/p' ./../biopac/S2_2016-05-11T10_16_44.txt>./../biopacBeforeEvent14s/S_2_23_.txt
sed -n '/^38927/,/^38941/p' ./../biopac/S2_2016-05-11T10_16_44.txt>./../biopacBeforeEvent14s/S_2_24_.txt
sed -n '/^38975/,/^38989/p' ./../biopac/S2_2016-05-11T10_16_44.txt>./../biopacBeforeEvent14s/S_2_25_.txt
sed -n '/^39036/,/^39050/p' ./../biopac/S2_2016-05-11T10_16_44.txt>./../biopacBeforeEvent14s/S_2_26_.txt
sed -n '/^40456/,/^40470/p' ./../biopac/S2_2016-05-11T1

In [83]:
# minimum interval between idividuals
a=dfAll.groupby('sujet').Absolute_sec.first()[1:]
b=dfAll.groupby('sujet').Absolute_sec.last()[:-1]
a=a.reset_index(drop=True)
b=b.reset_index(drop=True)
np.abs(a-b).min()

3933

In [84]:
# select no overlapping events
sepevent=dfAll.loc[~np.insert((np.diff(dfAll.iloc[:,2])<30)&(np.abs(np.diff(dfAll.iloc[:,2]))<3933),0,False)]

In [85]:
sepevent.head()

,sujet,match,Absolute_sec,event,emotions,arousal,valence
0,1,1,47637,rate,frustration,2,-1
1,1,1,47813,arret du gardien,colere,2,-1
2,1,1,47852,but,colere,2,-3
3,1,1,48157,tir,peur,1,-1
4,1,2,49632,arret du gardien,frustration,2,-2


## Event feature for each mi-temps

In [46]:
misep=pd.DataFrame()
for key, value in Mdf.iteritems():
    
    sujet=int(key[1::])
    match=1
    septime=value.iloc[4,3]
    sepl=pd.DataFrame([[sujet,match,septime]], columns=['sujet','match','septime'])
    misep=misep.append(sepl)
    
    sujet=int(key[1::])
    match=2
    septime=value.iloc[7,3]
    sepl=pd.DataFrame([[sujet,match,septime]], columns=['sujet','match','septime'])
    misep=misep.append(sepl)
    
    sujet=int(key[1::])
    match=3
    septime=value.iloc[10,3]
    sepl=pd.DataFrame([[sujet,match,septime]], columns=['sujet','match','septime'])
    misep=misep.append(sepl)
    
misep['sujet']=misep['sujet'].astype(str)
misep['match']=misep['match'].astype(str)   

dfAll['sujet']=dfAll['sujet'].astype(str)
dfAll['match']=dfAll['match'].astype(str) 

misepn=misep.set_index(['sujet','match'])
dfAlln=dfAll.set_index(['sujet','match'])


In [47]:
dfAlln1=dfAlln.join(misepn).dropna()
def mitempset(row):
    if row['Absolute_sec']< row['septime']:
        return '1'
    else:
        return '2'
dfAlln1['mitemps']=dfAlln1.apply(mitempset,axis=1)
dfAlln1=dfAlln1.reset_index()

In [48]:
dfAlln1.head()

,sujet,match,Absolute_sec,event,emotions,arousal,valence,niveau,septime,mitemps
0,10,1,59626,rate,frustration,0,-2,3,59946,1
1,10,1,59651,rate,colere,0,-1,3,59946,1
2,10,1,59757,tir,frustration,2,-1,3,59946,1
3,10,1,59809,rate,colere,1,-1,3,59946,1
4,10,1,59852,geste technique,colere,2,-2,3,59946,1


In [50]:
# Number of events related with "valence"
valencepos=dfAlln1.ix[dfAlln1['valence']>0,:].groupby(['sujet','match','mitemps']).size().to_frame('vapos')
valenceneg=dfAlln1.ix[dfAlln1['valence']<=0,:].groupby(['sujet','match','mitemps']).size().to_frame('vaneg')
fortvalence=dfAlln1.ix[dfAlln1['valence']==-3,:].groupby(['sujet','match','mitemps']).size().to_frame('fortva')
# Number of events related with "arousal"
arousalpos=dfAlln1.ix[dfAlln1['arousal']>0,:].groupby(['sujet','match','mitemps']).size().to_frame('arpos')
arousalneg=dfAlln1.ix[dfAlln1['arousal']<=0,:].groupby(['sujet','match','mitemps']).size().to_frame('arneg')
fortarousal=dfAlln1.ix[dfAlln1['arousal']==3,:].groupby(['sujet','match','mitemps']).size().to_frame('forar')
# Number of events related with emotions
numfrustration=dfAlln1.ix[dfAlln1['emotions']=='frustration',:].groupby(['sujet','match','mitemps']).size().to_frame('frus')
numcolere=dfAlln1.ix[dfAlln1['emotions']=='colere',:].groupby(['sujet','match','mitemps']).size().to_frame('colere')
numjoie=dfAlln1.ix[dfAlln1['emotions']=='joie',:].groupby(['sujet','match','mitemps']).size().to_frame('joie')
# Number of events in each match
allevent = dfAlln1.groupby(['sujet','match','mitemps']).size().to_frame('all')

EventFeature=allevent.join([valencepos,valenceneg,fortvalence,arousalpos,arousalneg,fortarousal,numfrustration,numcolere,numjoie])
EventFeature=EventFeature.fillna(0)
EventFeature=EventFeature.reset_index()
EventFeature['sujet']=EventFeature['sujet'].astype(int)
EventFeature=EventFeature.sort(['sujet','match','mitemps']).reset_index(drop=True)

In [51]:
EventFeature.head()
#EventFeature.to_csv('./../EventFeature.csv',encoding='utf-16',index=False)

,sujet,match,mitemps,all,vapos,vaneg,fortva,arpos,arneg,forar,frus,colere,joie
0,2,1,1,3,1,2,1,3,0,1,1,1,0
1,2,1,2,6,1,5,2,5,1,1,4,1,0
2,2,2,1,10,1,9,0,9,1,1,4,3,1
3,2,2,2,7,1,6,1,5,2,1,3,2,0
4,2,3,1,11,2,9,4,9,2,4,6,2,1


# Participant informations

In [ ]:
participant=pd.read_csv('./../out/MatchStatistiques.csv',sep=',')

In [7]:
dfAll = pd.read_csv('./../out/Allevent.txt',encoding='utf-16',index_col=0)
dfAll.head()

,sujet,match,Absolute_sec,event,emotions,arousal,valence
0,1,1,47637,rate,frustration,2.0,-1.0
1,1,1,47813,arret du gardien,colere,2.0,-1.0
2,1,1,47852,but,colere,2.0,-3.0
3,1,1,48157,tir,peur,1.0,-1.0
4,1,2,49632,arret du gardien,frustration,2.0,-2.0


# Plot the whole sequence

In [9]:
def ploteventtrait(plt, eventtmp):
    for index, row in eventtmp.iterrows():
        #mywidth=np.fabs(row['arousal'])+1
        if row['arousal']>0:
            pos_down = 0.5
            pos_up= 0.5 + 0.15 * row['arousal']
        else:
            pos_down = 0.5 + 0.15*row['arousal']-0.02
            pos_up= 0.5
        plt.axvline(row['rela_point']-300,color='r',ymin=pos_down, ymax=pos_up,linewidth=2)
        
    for index, row in eventtmp.iterrows():
        #mywidth=np.fabs(row['valence'])+1
        if row['valence']>0:
            pos_down = 0.5
            pos_up= 0.5 + 0.15 * row['valence']
        else:
            pos_down = 0.5 + 0.15*row['valence']-0.02
            pos_up= 0.5
        plt.axvline(row['rela_point']+300,color='g',ymin=pos_down, ymax=pos_up,linewidth=2)   
    

def plotseq(chunkdf,sujet, sequence,eventtmp):
    fig = plt.figure(figsize=(30,20))
    ns,nm=subject_level(sujet,sequence)
    figurename='S'+str(sujet)+'_'+sequence
    fig.suptitle('S'+str(sujet)+'_'+sequence + ': sujetlevel '+str(ns)+'_matchlevel '+str(nm), fontsize=30)
    gs = gridspec.GridSpec(7,1)
     # raw signal

    ax1 = fig.add_subplot(gs[0, 0])
    ax1.plot(range(l), chunkdf['ecg'], linewidth=1, label='ecg')
    #ax1.legend()
    ax1.set_ylabel('ECG')
    ploteventtrait(plt,eventtmp)

    ax2 = fig.add_subplot(gs[1, 0], sharex=ax1)
    ax2.plot(range(l), chunkdf['rsp'], linewidth=1, label='respiration')
    #ax2.legend()
    ax2.set_ylabel('Resp')
    ploteventtrait(plt,eventtmp)

    ax3 = fig.add_subplot(gs[2, 0], sharex=ax1)
    ax3.plot(range(l), chunkdf['eda'], linewidth=1, label='eda')
    #ax3.legend()
    ax3.set_ylabel('EDA')
    ploteventtrait(plt,eventtmp)

    ax4 = fig.add_subplot(gs[3, 0], sharex=ax1)
    ax4.plot(range(l), chunkdf['skt'], linewidth=1, label='skinTemp')
    #ax4.legend()
    ax4.set_ylabel('SkinTemp')
    ploteventtrait(plt,eventtmp)

    ax5 = fig.add_subplot(gs[4, 0], sharex=ax1)
    ax5.plot(range(l), chunkdf['emg_s'], linewidth=1, label='emg_s')
    #ax5.legend()
    ax5.set_ylabel('EMG_m')
    ploteventtrait(plt,eventtmp)

    ax6 = fig.add_subplot(gs[5, 0], sharex=ax1)
    ax6.plot(range(l), chunkdf['emg_f'], linewidth=1, label='emg_f')
    #ax6.legend()
    ax6.set_ylabel('EMG_f')
    ploteventtrait(plt,eventtmp)

    ax7 = fig.add_subplot(gs[6, 0], sharex=ax1)
    ax7.plot(range(l), chunkdf[['x','y','z']], linewidth=1)
    #ax7.legend(['x','y','z'])
    ax7.set_ylabel('Acce')
    ploteventtrait(plt,eventtmp)

    savename='./../plot/'+figurename+'.pdf'
    fig.savefig(savename,dpi=100)
    #plt.show()
    plt.close()
def subject_level(sujet,sequence):
    mbf=pd.read_csv('./../out/matbutFeature.csv',encoding='utf-16')
    sdf=mbf.groupby(['sujet','match']).first().reset_index()
    ns=int(sdf.loc[(sdf['sujet']==sujet) & (sdf['match']==int(sequence[1])),'niveausub'])
    nm=int(sdf.loc[(sdf['sujet']==sujet) & (sdf['match']==int(sequence[1])),'niveaumat'])
    return (ns,nm)

In [83]:
chunkdf=pd.read_csv('/home/wenlu/Documents/XPFIFA/biopacEvent/S_2_21_.txt',sep='\t',header=None,usecols=range(0,11),names=['ts','rsp', 'x', 'emg_s','skt','y','eda','ecg','z','emg_f','temp'])
l=len(chunkdf)
sujet = 2
sequence = 'event'
#Get beginning timestamp (Absolute_sec) of selected sequence
strsujet = 'S'+str(sujet)
eventtmp =pd.DataFrame()
plotseq(chunkdf,sujet,sequence,eventtmp)

In [10]:
sujet = 19
sequence = 'm12'
bioMatchPath="./../biopacMatch/"
#Read Biopac files of selected sequence : exp: ./../biopacMatch/S4_m12.txt
matchFilename=bioMatchPath+"S"+str(sujet)+"_"+sequence+".txt"
print matchFilename
chunkdf=pd.read_csv(matchFilename,sep='\t',nrows=240000,header=None,usecols=range(0,11),names=['ts','rsp', 'x', 'emg_s','skt','y','eda','ecg','z','emg_f','temp'])
l=len(chunkdf)

#Get beginning timestamp (Absolute_sec) of selected sequence
strsujet = 'S'+str(sujet)
begin = Mdf[strsujet].ix[sequence,6]
end = Mdf[strsujet].ix[sequence,4]

#Get event list of selected sequence
eventtmp=dfAll.ix[(dfAll['sujet']==sujet) ,:]
eventtmp=eventtmp[(eventtmp.Absolute_sec>begin)&(eventtmp.Absolute_sec<end)]
eventtmp['rela_point']=pd.Series((eventtmp.Absolute_sec-begin)*1000) #event point respect to the beginning timestamp
#eventtmp
plotseq(chunkdf,sujet,sequence,eventtmp)

./../biopacMatch/S19_m12.txt


In [24]:
eventtmp

,sujet,match,Absolute_sec,event,emotions,arousal,valence,rela_point
563,19,1,60849,but,fierte,3.0,3.0,35000
564,19,1,60927,rate,joie,2.0,1.0,113000
565,19,1,60946,tacle,frustration,2.0,-2.0,132000
566,19,1,60963,arret du gardien,neutre,2.0,-1.0,149000
567,19,1,60999,faute,frustration,1.0,-1.0,185000
568,19,1,61034,tir,peur,2.0,0.0,220000


#### Draw all

In [7]:
bioMatchPath="./../biopacMatch/"
for fn in os.listdir(bioMatchPath):
    sujet=re.split('_|\.|S', fn)[1]
    sequence=re.split('_|\.|S', fn)[2]
    matchFilename=bioMatchPath+fn
    #print matchFilename
    chunkdf=pd.read_csv(matchFilename,sep='\t',nrows=240000,header=None,usecols=range(0,11),names=['ts','rsp', 'x', 'emg_s','skt','y','eda','ecg','z','emg_f','temp'])
    l=len(chunkdf)
    #Get beginning timestamp (Absolute_sec) of selected sequence
    strsujet = 'S'+str(sujet)
    begin = Mdf[strsujet].ix[sequence,6]
    end = Mdf[strsujet].ix[sequence,4]

    #Get event list of selected sequence
    eventtmp=dfAll.ix[(dfAll['sujet']==int(sujet)) ,:]
    eventtmp=eventtmp[(eventtmp.Absolute_sec>begin)&(eventtmp.Absolute_sec<end)]
    eventtmp['rela_point']=pd.Series((eventtmp.Absolute_sec-begin)*1000) #event point respect to the beginning timestamp
    #eventtmp
    #print matchFilename, len(eventtmp)
    plotseq(chunkdf,sujet,sequence,eventtmp)
    savename='./../out/plot/'+str(eventidx)+'_S_'+str(sujet)+'.pdf'
    plt.savefig(savename)
    plt.close(fig)

NameError: name 'Mdf' is not defined

In [11]:
#draw all in order
dfAll = pd.read_csv('./../out/Allevent.txt',encoding='utf-16',index_col=0)
bioMatchPath="./../biopacMatch/"
for sujet in range(2,59):
    for sequence in ['m11','m12','m21','m22','m31','m32']:
        fn='S'+str(sujet)+'_'+sequence+'.txt'
        matchFilename=bioMatchPath+fn
        #print fn
        try:
            chunkdf=pd.read_csv(matchFilename,sep='\t',nrows=240000,header=None,usecols=range(0,11),names=['ts','rsp', 'x', 'emg_s','skt','y','eda','ecg','z','emg_f','temp'])
        except:
            print fn+' not found'
            continue
        #Get beginning timestamp (Absolute_sec) of selected sequence
        strsujet = 'S'+str(sujet)
        begin = Mdf[strsujet].ix[sequence,6]
        end = Mdf[strsujet].ix[sequence,4]

        #Get event list of selected sequence
        eventtmp=dfAll.ix[(dfAll['sujet']==int(sujet)) ,:]
        eventtmp=eventtmp[(eventtmp.Absolute_sec>begin)&(eventtmp.Absolute_sec<end)]
        eventtmp['rela_point']=pd.Series((eventtmp.Absolute_sec-begin)*1000) #event point respect to the beginning timestamp
        #eventtmp
        #print matchFilename, len(eventtmp)
        plotseq(chunkdf,sujet,sequence,eventtmp)
'''
for sujet in range(1,59):
    for sequence in ['base0','base1','base2','base3']:
        fn='S'+str(sujet)+'_'+sequence+'.txt'
        matchFilename=bioMatchPath+fn
        #print fn
        try:
            chunkdf=pd.read_csv(matchFilename,sep='\t',nrows=180000,header=None,usecols=range(0,11),names=['ts','rsp', 'x', 'emg_s','skt','y','eda','ecg','z','emg_f','temp'])
        except:
            print fn+' not found'
            continue
        #Get beginning timestamp (Absolute_sec) of selected sequence
        strsujet = 'S'+str(sujet)
        begin = Mdf[strsujet].ix[sequence,6]
        end = Mdf[strsujet].ix[sequence,4]

        #Get event list of selected sequence
        eventtmp=dfAll.ix[(dfAll['sujet']==int(sujet)) ,:]
        eventtmp=eventtmp[(eventtmp.Absolute_sec>begin)&(eventtmp.Absolute_sec<end)]
        eventtmp['rela_point']=pd.Series((eventtmp.Absolute_sec-begin)*1000) #event point respect to the beginning timestamp
        #eventtmp
        #print matchFilename, len(eventtmp)
        plotseq(chunkdf,sujet,sequence,eventtmp)
'''       

"\nfor sujet in range(1,59):\n    for sequence in ['base0','base1','base2','base3']:\n        fn='S'+str(sujet)+'_'+sequence+'.txt'\n        matchFilename=bioMatchPath+fn\n        #print fn\n        try:\n            chunkdf=pd.read_csv(matchFilename,sep='\t',nrows=180000,header=None,usecols=range(0,11),names=['ts','rsp', 'x', 'emg_s','skt','y','eda','ecg','z','emg_f','temp'])\n        except:\n            print fn+' not found'\n            continue\n        #Get beginning timestamp (Absolute_sec) of selected sequence\n        strsujet = 'S'+str(sujet)\n        begin = Mdf[strsujet].ix[sequence,6]\n        end = Mdf[strsujet].ix[sequence,4]\n\n        #Get event list of selected sequence\n        eventtmp=dfAll.ix[(dfAll['sujet']==int(sujet)) ,:]\n        eventtmp=eventtmp[(eventtmp.Absolute_sec>begin)&(eventtmp.Absolute_sec<end)]\n        eventtmp['rela_point']=pd.Series((eventtmp.Absolute_sec-begin)*1000) #event point respect to the beginning timestamp\n        #eventtmp\n        #pr

# Calculate features for each event file

In [3]:
import numpy as np
from scipy.interpolate import interp1d
from biosppy.signals import tools as st
from biosppy.signals import eda
from biosppy.signals import ecg
from biosppy.signals import emg
from biosppy.signals import resp
from biosppy.signals import tools as st
from biosppy import plotting, utils
from scipy.stats import stats


In [49]:
def feature_all(chunkdf,sujet, sequence):
    if sequence=='en1':
        NumMatch=0
        Nummitemp=1
    elif sequence=='en2':
        NumMatch=0
        Nummitemp=2
    elif sequence[0]=='b':
        NumMatch=5
        Nummitemp=int(sequence[-1])
    else:
        NumMatch=int(sequence[1])
        Nummitemp=int(sequence[2])
        
    seq_key=['sujet','sequence','NumMatch' ,'Nummitemp']
    seq_info=(sujet,sequence,NumMatch,Nummitemp)
    ecg_key, ecg_feature=ecgfeature(chunkdf['ecg'])
    emgf_key, emgf_feature=emgfeature(chunkdf['emg_f'])
    emgf_key=[i+'_f' for i in emgf_key]
    emgs_key, emgs_feature=emgfeature(chunkdf['emg_s'])
    emgs_key=[i+'_s' for i in emgf_key]
    eda_key, eda_feature=edafeature(chunkdf['eda'])
    resp_key, resp_feature=respfeature(chunkdf['rsp'])
    
    key = seq_key + ecg_key + emgf_key + emgs_key + eda_key + resp_key
    feature= seq_info + ecg_feature + emgf_feature + emgs_feature + eda_feature + resp_feature
    return pd.DataFrame([list(feature)], columns=key)

In [50]:
def feature_event(chunkdf,sujet, sequence):
    seq_key=['sujet','sequence']
    seq_info=(sujet,sequence)
    
    ecg_key, ecg_feature=ecgfeature(chunkdf['ecg'])
    
    emgf_key, emgf_feature=emgfeature(chunkdf['emg_f'])
    emgf_key=[i+'_f' for i in emgf_key]
    emgs_key, emgs_feature=emgfeature(chunkdf['emg_s'])
    emgs_key=[i+'_s' for i in emgf_key]
    
    eda_key, eda_feature=edafeature(chunkdf['eda'])
    
    resp_key, resp_feature=respfeature(chunkdf['rsp'])
    
    key = seq_key + ecg_key + emgf_key + emgs_key + eda_key + resp_key
    feature= seq_info + ecg_feature + emgf_feature + emgs_feature + eda_feature + resp_feature
    return pd.DataFrame([list(feature)], columns=key)

In [53]:
chunkdf=pd.read_csv('./../biopacMatch/S18_en2.txt',sep='\t',nrows=240000,header=None,usecols=[0,1,3,6,7,9],names=['ts','rsp', 'emg_s','eda','ecg','emg_f'])


In [83]:
PhysioMat_feature = pd.DataFrame()
bioMatchPath="./../biopacMatch/"
sampling_rate=1000
for fn in os.listdir(bioMatchPath):
    print fn
    sujet=re.split('_|\.|S', fn)[1]
    sequence=re.split('_|\.|S', fn)[2]
    if sequence[0]=='b':
        nr=180000
    else:
        nr=240000
    matchFilename=bioMatchPath+fn
    print (matchFilename,sujet,sequence)
    chunkdf=pd.read_csv(matchFilename,sep='\t',nrows=nr,header=None,usecols=[0,1,3,6,7,9],names=['ts','rsp', 'emg_s','eda','ecg','emg_f'])
    print "read ok"
    print len(chunkdf)
    tmpdf=feature_all(chunkdf,sujet,sequence)
    print (matchFilename,sujet,sequence)
    PhysioMat_feature=PhysioMat_feature.append(tmpdf, ignore_index=True)
    


S16_base0.txt
('./../biopacMatch/S16_base0.txt', '16', 'base0')
read ok
180000
(34, 34)
(1, 101)
(1, 101)
(34, 234)
('./../biopacMatch/S16_base0.txt', '16', 'base0')
S18_en2.txt
('./../biopacMatch/S18_en2.txt', '18', 'en2')
read ok
240000
(34, 34)
(1, 101)
(1, 101)
(34, 234)
('./../biopacMatch/S18_en2.txt', '18', 'en2')
S33_base3.txt
('./../biopacMatch/S33_base3.txt', '33', 'base3')
read ok
180000
(34, 34)
(1, 101)
(1, 101)
(34, 234)
('./../biopacMatch/S33_base3.txt', '33', 'base3')
S48_m12.txt
('./../biopacMatch/S48_m12.txt', '48', 'm12')
read ok
240000
(34, 34)
(1, 101)
(1, 101)
(34, 234)
('./../biopacMatch/S48_m12.txt', '48', 'm12')
S52_base3.txt
('./../biopacMatch/S52_base3.txt', '52', 'base3')


KeyboardInterrupt: 

In [57]:
PhysioMat_feature=PhysioMat_feature.sort(['sujet','NumMatch','Nummitemp']).reset_index(drop=True) 
PhysioMat_feature.to_csv('./../physioFeature.csv',encoding='utf-16')

## Normalized physio feature

In [180]:
# Get feature dataframe
PhysioMat_feature=pd.read_csv('/home/wenlu/Documents/XPFIFA/PhysioMatFeature.csv',encoding='utf-16',index_col=0)
# Change columns structure for groupby & transform: feature0, feature1, ... , sujet
dffeature=PhysioMat_feature.iloc[:,4::]
dffeature['sujet']=PhysioMat_feature.iloc[:,0]
# Normalize dataframe for each subject using Transform 
featureTrans=dffeature.groupby('sujet').transform(lambda x: (x - x.mean()) / x.std())
# Concat sequence info with normalized features
PhysioMatFeatureNorm=pd.concat([PhysioMat_feature.iloc[:,0:4], featureTrans], axis=1)
PhysioMatFeatureNorm.to_csv('./../PhysioMatFeatureNorm.csv',encoding='utf-16')

PhysioMatFeatureNorm.head()

,sujet,sequence,NumMatch,Nummitemp,hbd_mean,hbd_median,hbd_max,hbd_min,hbd_vrange,hbd_var,...,rsp_b0,rsp_b1,rsp_b2,rsp_b3,rsp_b4,rsprt_b0,rsprt_b1,rsprt_b2,rsprt_b3,rsprt_b4
0,2,en1,0,1,-0.687879,-0.660910,-0.089296,-1.275605,0.689518,-0.020712,...,-0.303359,-0.303437,-0.266176,0.168155,0.006136,-1.343083,-1.343083,NaN,NaN,NaN
1,2,en2,0,2,-0.867809,-0.854349,-0.426662,-0.149448,-0.473627,-0.844491,...,-0.711939,-0.759770,-0.560952,-0.742660,-0.605644,0.294588,0.294588,NaN,NaN,NaN
2,2,m11,1,1,-0.296863,-0.246397,-0.783873,-0.232867,-0.896590,-0.582471,...,-0.746706,-0.745034,-0.637209,-0.555939,-0.757266,1.276979,1.276979,NaN,NaN,NaN
3,2,m12,1,2,-0.202853,-0.218763,-0.287746,-0.107739,-0.315017,-0.848462,...,-0.765506,-0.735363,-0.670909,-0.901918,-0.986010,1.078812,1.078812,NaN,NaN,NaN
4,2,m21,2,1,0.571999,0.610263,0.188535,0.935000,-0.341452,-0.093550,...,-0.624521,-0.655252,-0.514691,-0.005012,-0.393081,0.372294,0.372294,NaN,NaN,NaN


## Event physio feature calculation

In [ ]:
Event_feature = pd.DataFrame()
bioEventPath="./../biopacEvent/"
for fn in os.listdir(bioEventPath):
    sujet=fn.split('_')[1]
    sequence=fn.split('_')[2]
    matchFilename=bioEventPath+fn
    #print (matchFilename,sujet,sequence)
    chunkdf=pd.read_csv(matchFilename,sep='\t',nrows=30000,header=None,usecols=[0,1,3,6,7,9],names=['ts','rsp', 'emg_s','eda','ecg','emg_f'])
    tmpdf=feature_event(chunkdf,sujet,sequence)
    print (matchFilename,sujet,sequence)
    Event_feature=Event_feature.append(tmpdf, ignore_index=True)
